In [1]:
import pandas as pd 
import time
import requests
import datetime


In [4]:
file=pd.read_csv(r'Google DS\General_raw_data\CrX7kMhLC3cSsXJdT7JDgqrRVWGnUpX3gfEfxxU2NVLi_2022-01-18.csv')

In [5]:
stsol=file[file['token']=='7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj']
stsol['blocktime'] = pd.to_datetime(stsol['blocktime'],unit='s')
stsol['blocktime'] = stsol['blocktime'].dt.floor('d')
stsol_grouped=stsol.groupby('blocktime')['change'].sum().reset_index()
stsol_grouped['cum_sum']=stsol_grouped['change'].cumsum()

In [11]:
stsol_grouped['incentives']=0
stsol_grouped.loc[stsol_grouped['blocktime']>'2021-12-15','incentives']=5666.66
token1_prices=pd.read_csv(r'Prices\stSOL_prices.csv')
token1_prices['blocktime']=pd.to_datetime(token1_prices['blocktime'],infer_datetime_format=True)
new_names={'blocktime':'blocktime',"Price":"stSOL_price"}
token1_prices.rename(columns=new_names, inplace=True)


In [93]:
token2_prices=pd.read_csv(r'Prices\LDO_prices.csv')
token2_prices['blocktime']=pd.to_datetime(token2_prices['blocktime'],infer_datetime_format=True)
new_names={'blocktime':'blocktime',"Price":"incentive_price"}
token2_prices.rename(columns=new_names, inplace=True)
token2_prices=token2_prices.sort_values(by='blocktime',ascending=True,ignore_index=True)


    blocktime     Price
0  2021-10-12  3.805051
1  2021-10-13  3.536080
2  2021-10-14  3.725558
3  2021-10-15  4.139435
4  2021-10-16  4.033364
5  2021-10-17  3.737817
6  2021-10-18  3.558273
7  2021-10-19  3.295390
8  2021-10-20  3.413604
9  2021-10-21  3.806661


In [95]:
prices=pd.merge(token1_prices,token2_prices,on='blocktime',how='inner')
final_table=stsol_grouped.merge(prices,on='blocktime',how='left')
final_table['stSOL_usd']=final_table['change']*final_table['stSOL_price']
final_table['incentives_usd']=final_table['incentives']*final_table['incentive_price']
final_table.to_csv(r'Dashboard_data\Google DS\General\stSOL_info.csv',sep=',',mode='a',header=False, encoding='utf-8',index=False)